# Import Necessary Libraries

In [33]:
import pickle
import numpy as np
import networkx as nx
import os
import shutil
import math
from random import shuffle
import pprint
pp = pprint.PrettyPrinter()

#### Importing chinese whispers algorithm
As we have already defined the clustering algorithm in `clusterer.py` we can directly import it

In [2]:
from clusterer import chineseWhispers

The embeddings for the faces in the dataset (LFW in this case) should be loaded. The embeddings can be computed from the script
`embedder.py`

# Run Chinese Whispers on the embeddings

In [4]:
data = pickle.load(open("embeddings_pool.pickle","rb"))

Since the embeddings are loaded we can then create the graph

In [ ]:
graph = chineseWhispers(data,0.8,20)
# Takes about 5 minutes for this dataset

# Calculate Evaluation Metric

The evaluation metric used is the F-score.  
F-score is defined as the harmonic mean of precision and recall.

F-score = (2*precison*recall)/(precision + recall) 
(Replace with latex later)

We will first calculate the precision and recall using True positives, False positives and False negatives

In [34]:
# Define function for nCr
def nCr(n,r):
    fact = math.factorial
    return fact(n)/(fact(r)*fact(n-r))

def partial_dict_view(dictionary,n):
    pp.pprint({k: v for i, (k, v) in enumerate(dictionary.items()) if i < n})

Firstly create a dictionary which maps a cluster to the number of images it contains.  
**NOTE:** a lot of people in the LFW dataset have only one image of them, hence there can be many clusters with only one image

In [10]:
cluster_to_num_images = {}
for node in graph.nodes:
    if graph.nodes[node]['pseudoClass'] in cluster_to_num_images:
        cluster_to_num_images[graph.nodes[node]['pseudoClass']] += 1
    else:
        cluster_to_num_images[graph.nodes[node]['pseudoClass']] = 1

In [15]:
partial_dict_view(cluster_to_num_images,5)

{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 7: 4}


Now let's compute the total positives
(add explanation later)

In [41]:
total_pw_positives = 0
for cluster,num_images in cluster_to_num_images.items():
    # It's a positive only if a pair can be formed
    if num_images >= 2:
        total_pw_positives += nCr(num_images,2)

In [42]:
print(total_pw_positives)

345969.0


- The subdirectory of the image in which it resides in the LFW folder is the identity of the person. 
- In this scenario the identity is the class.
- Hence the  classes can be taken from the subdirectory in the path of the image.
- The nodes of the networkx graph contain an attribute which has the relative path for the image.
- For example `lfw\\Aaron_Eckhart\\Aaron_Eckhart_0001.jpg` is the image corresponding to the person named Aaron Eckhart

Firstly we create a dictionary which maps a cluster to a dictionary which describes the cluster. i.e the latter's value is a dictionary which maps each class (identity) in the cluster to the number of nodes (image embeddings) in the cluster

In [57]:
clusters = list(cluster_to_num_images.keys())

In [27]:
cluster_to_desc = {}
for cluster in clusters:
    cluster_to_desc[cluster] = {}
    for node in graph.nodes:
        if graph.nodes[node]['pseudoClass'] == cluster:
            
            Class = os.path.split(graph.nodes[node]['path'])[0]
            Class = Class[4:]
            # find a cleaner implementation
            if Class not in cluster_to_desc[cluster]:
                cluster_to_desc[cluster][Class] = 1
            else:
                cluster_to_desc[cluster][Class] += 1

In [35]:
# Detailed description of the dictionary
partial_dict_view(cluster_to_desc,10)

{1: {'AJ_Cook': 1},
 2: {'AJ_Lamas': 1},
 3: {'Aaron_Eckhart': 1},
 4: {'Aaron_Guiel': 1},
 5: {'Aaron_Patterson': 1},
 7: {'Aaron_Peirsol': 4},
 12: {'Aaron_Sorkin': 2},
 5040: {'Abba_Eban': 1,
        'Bernard_Landry': 1,
        'Bobby_Bowden': 1,
        'Cass_Ballenger': 1,
        'Charles_Chandler_IV': 1,
        'Donald_Keck': 1,
        'Edward_Egan': 1,
        'Gene_Robinson': 1,
        'George_Brumley': 1,
        'Hans_Blix': 38,
        'Ignacio_Antonio_Velasco': 1,
        'Itzhak_Perlman': 1,
        'Jesse_Helms': 1,
        'Jim_Talent': 1,
        'Kenneth_Reichert': 1,
        'Larry_Tanenbaum': 1,
        'Michael_J_Sheehan': 2,
        'Miguel_Aldana_Ibarra': 1,
        'Pedro_Malan': 2,
        'Ray_Bradbury': 1,
        'Robert_Gordon_Card': 1,
        'Rupert_Murdoch': 1,
        'Sidney_Kimmel': 1},
 8682: {'Aaron_Tippin': 1, 'Marwan_Barghouthi': 2},
 10467: {'Aaron_Pena': 1,
         'Aram_Adler': 1,
         'Martin_Torrijos': 1,
         'Ray_Romano': 9}}


Now we can compute the true positives. (give explanation later)

In [43]:
true_pw_positives = 0
for cluster in cluster_to_desc.keys():
    for Class in cluster_to_desc[cluster].keys():
        if cluster_to_desc[cluster][Class] >= 2:
            true_pw_positives += nCr(cluster_to_desc[cluster][Class],2)

In [44]:
print(true_pw_positives)

236091.0


False positives can then be calculated from subtracting from the total

In [45]:
false_pw_positives = total_pw_positives - true_pw_positives
print(false_pw_positives)

109878.0


Now let's create a dictionary which maps a class(identity) to the number of images it contains. 

In [68]:
class_to_num_images = {}
for Class in os.listdir("lfw"):
    num_images = 0
    for image in os.listdir(os.path.join("lfw",Class)):
        num_images += 1
    class_to_num_images[Class] = num_images

In [69]:
partial_dict_view(class_to_num_images,10)

{'Aaron_Eckhart': 1,
 'Aaron_Guiel': 1,
 'Aaron_Patterson': 1,
 'Aaron_Peirsol': 4,
 'Aaron_Pena': 1,
 'Aaron_Sorkin': 2,
 'Aaron_Tippin': 1,
 'Abba_Eban': 1,
 'Abbas_Kiarostami': 1,
 'Abdel_Aziz_Al-Hakim': 1}


We can then calculate the false negatives using the above dictionary

In [72]:
false_pw_negatives = 0
# Iterate through all classes
for Class in class_to_num_images.keys():
    prev_occurence = 0
    for cluster in cluster_to_num_images.keys():
        if Class in cluster_to_desc[cluster]:
            # Get the number of nodes for the current class in the cluster
            num_of_class_in_cluster = cluster_to_desc[cluster][Class]
            
            # Get the number of nodes for the current class not in the cluster
            # It can be calculated by subtracting the number of times a class occurs in the cluster
            # from the total number of times it occurs in the dataset
            num_of_class_out_of_cluster = class_to_num_images[Class] - num_of_class_in_cluster
            
            # The number of pairs formed containing mismatched nodes is the number of nodes in
            # the cluster multiplied with the number out of the cluster
            # To account for pairs added by the previous cluster we subtract 'prev'
            false_pw_negatives += num_of_class_in_cluster*(num_of_class_out_of_cluster-prev_occurence)
            prev_occurence += num_of_class_in_cluster
            

In [74]:
print(false_pw_negatives)

6166
